In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

Metodo che serve a prelevare tutti i titoli e i sotto-link da una singola pagina e li salva su imm_data

In [2]:
url = "https://www.immobiliare.it/vendita-case/firenze/"
page = requests.get(url)

In [3]:
url = "https://www.immobiliare.it/vendita-case/firenze/"
page = requests.get(url)

In [4]:
soup = BeautifulSoup(page.content, "html.parser")
results = soup.find("ul", {"class": "nd-list in-realEstateResults"})
houses = results.find_all("li", class_="nd-list__item in-realEstateResults__item")

In [5]:
data = []

for i in range(0, len(houses)):
    house_detail = houses[i].find("div", class_='nd-mediaObject__content in-card__content in-realEstateListCard__content')

    tmp = []

    a_href=house_detail.find("a",{"class":"in-card__title"}).get("href")
    title=house_detail.find("a",{"class":"in-card__title"}).get("title")

    tmp = [title, a_href]

    data.append(tmp)

imm_data = pd.DataFrame(data, columns=['Title', 'Link'])

In [6]:
column_name = []

Metodo che serve per prelevare i nomi che avranno le varie colonne e li salva sulla lista column_name

In [7]:
for index, el in imm_data.iterrows():
    url_detail = el.Link
    page_detail = requests.get(url_detail)
    soup_detail = BeautifulSoup(page_detail.content, "html.parser")
    results = soup_detail.find("section", {"class": "in-wrapper is-detailView in-landingDetail"})

    main_feature = results.find_all("dl", class_="in-realEstateFeatures__list")
    
    for i in range(0, len(main_feature)):
        title_feature = main_feature[i].find_all("dt", class_="in-realEstateFeatures__title")
        for el in title_feature:
            if str(el.text) not in column_name:
                column_name.append(el.text)
            

In [8]:
column_name

['Riferimento e Data annuncio',
 'contratto',
 'tipologia',
 'superficie',
 'locali',
 'piano',
 'totale piani edificio',
 'Posti Auto',
 'disponibilità',
 'altre caratteristiche',
 'prezzo',
 'spese condominio',
 'anno di costruzione',
 'stato',
 'riscaldamento',
 'Prestazione energetica del fabbricato',
 'Efficienza energetica',
 'Indice prest. energetica rinnovabile',
 'Climatizzatore',
 'certificazione energetica']

In [9]:
column_name.insert(0, 'url')
column_name.insert(1, 'titolo')
column_name.insert(2, 'comune')
column_name.insert(3, 'quartiere')
column_name.insert(4, 'indirizzo')
column_name.insert(5, 'num_vani')
column_name.insert(6, 'superficiem^2')
column_name.insert(7, 'bagni')


In [10]:
column_name

['url',
 'titolo',
 'comune',
 'quartiere',
 'indirizzo',
 'num_vani',
 'superficiem^2',
 'bagni',
 'Riferimento e Data annuncio',
 'contratto',
 'tipologia',
 'superficie',
 'locali',
 'piano',
 'totale piani edificio',
 'Posti Auto',
 'disponibilità',
 'altre caratteristiche',
 'prezzo',
 'spese condominio',
 'anno di costruzione',
 'stato',
 'riscaldamento',
 'Prestazione energetica del fabbricato',
 'Efficienza energetica',
 'Indice prest. energetica rinnovabile',
 'Climatizzatore',
 'certificazione energetica']

In [11]:
detail_house_dict = {}

for i in column_name:
    detail_house_dict[i] = None

In [12]:
df = pd.DataFrame(columns=column_name)
#df.info()

In [13]:

for index, el in imm_data.iterrows():
    url_detail = el.Link
    #print(url_detail)
    
    detail_house_dict['url'] = el.Link
    detail_house_dict['titolo'] = el.Title
    
    
    page_detail = requests.get(url_detail)
    soup_detail = BeautifulSoup(page_detail.content, "html.parser")
    
    location = soup_detail.find("a", {"class": "in-titleBlock__link"}) 
    location_detail = location.find_all("span", class_="in-location")
        
    for i in range (0, 3):
        if i == 0 and len(location_detail) >= 1:
            detail_house_dict['comune'] = location_detail[0].text
        if i == 1 and len(location_detail) >= 2:
            detail_house_dict['quartiere'] = location_detail[1].text
        if i == 2 and len(location_detail) >= 3:
            detail_house_dict['indirizzo'] = location_detail[2].text
            
    results = soup_detail.find("ul", {"class", "nd-list nd-list--pipe in-feat in-feat--full in-feat__mainProperty in-landingDetail__mainFeatures"})
    main_feature = results.find_all("li", class_="nd-list__item in-feat__item")

    for el in main_feature:
        if(el.get("aria-label") == 'locali'):
            detail_house_dict['num_vani'] = el.text
        elif(el.get("aria-label") == 'superficie'):
            detail_house_dict['superficiem^2'] = el.text
        elif(el.get("aria-label") == 'bagno'):
            detail_house_dict["bagni"] = el.text
        elif(el.get("aria-label") == 'asd'):
            detail_house_dict["bagni"] = el.text
            
    results = soup_detail.find("section", {"class": "in-wrapper is-detailView in-landingDetail"})
    house_detail = results.find_all("dl", class_="in-realEstateFeatures__list")
    
    for i in range(0, len(house_detail)):
        for j in range(0, (len(house_detail[i])//2)):
            title_feature = house_detail[i].find_all("dt", class_="in-realEstateFeatures__title")[j:j+1]
            value_feature = house_detail[i].find_all("dd", class_="in-realEstateFeatures__value")[j:j+1]
            
            detail_house_dict[title_feature[0].text] = value_feature[0].text
    
    # Convertire il dictionary in list portando solo i 'valori' nell'ordine
    lst_value = []
    
    for dh in detail_house_dict:
        lst_value.append(detail_house_dict[dh])
        
    df.loc[len(df)] = lst_value

    #Ripulisco il dictionary per il giro successivo
    for i in column_name:
        detail_house_dict[i] = None


to create dictionary from tmp_value and tmp_title

In [14]:
df.head()

,url,titolo,comune,quartiere,indirizzo,num_vani,superficiem^2,bagni,Riferimento e Data annuncio,contratto,...,prezzo,spese condominio,anno di costruzione,stato,riscaldamento,Prestazione energetica del fabbricato,Efficienza energetica,Indice prest. energetica rinnovabile,Climatizzatore,certificazione energetica
0,https://www.immobiliare.it/annunci/102208708/,"Quadrilocale via Prospero Alpino, San Marcelli...",Firenze,San Marcellino,Via Prospero Alpino,4,99m²,1,37551001-269 - 01/04/2023,Vendita,...,€ 350.000,€ 110/mese,1965,Ottimo / Ristrutturato,"Autonomo, a radiatori, alimentato a metano",invernale estivo,E77 kWh/m² anno,None,None,None
1,https://www.immobiliare.it/annunci/102208746/,"Attico viale Matteotti, 11, San Marco - SS Ann...",Firenze,San Marco - SS Annunziata,"Viale Matteotti, 11",3,140m²,None,415V - 31/03/2023,Vendita,...,€ 980.000,Nessuna spesa condominiale,1900,Ottimo / Ristrutturato,None,None,F150 kWh/m² anno,50kWh/m² \n anno\n ...,None,None
2,https://www.immobiliare.it/annunci/102205958/,"Bilocale via del Ghirlandaio, Beccaria - Colom...",Firenze,Beccaria - Colombo,Via del Ghirlandaio,2,52m²,1,Arno - 31/03/2023,Vendita,...,€ 239.000,€ 70/mese,1900,Ottimo / Ristrutturato,"Centralizzato, a radiatori, alimentato a metano",None,F≥ 175 kWh/m² anno,None,Predisposizione impianto,None
3,https://www.immobiliare.it/annunci/102187476/,"Quadrilocale via degli Orti Oricellari, Borgo ...",Firenze,Borgo Ognissanti - Il Prato,Via degli Orti Oricellari,4,85m²,None,M200201 - 31/03/2023,Vendita,...,€ 550.000,€ 100/mese,1900,Ottimo / Ristrutturato,"Autonomo, a radiatori, alimentato a gas",None,A3.51 kWh/m² anno,None,Predisposizione impianto,None
4,https://www.immobiliare.it/annunci/102189186/,"Quadrilocale via Gaetano Milanesi, Leopoldo - ...",Firenze,Leopoldo - Vittorio Emanuele - Statuto,Via Gaetano Milanesi,4,112m²,None,A 404 - 31/03/2023,Vendita,...,€ 375.000,€ 50/mese,1930,Ottimo / Ristrutturato,"Autonomo, a radiatori, alimentato a metano",None,G169 kWh/m² anno,None,"Autonomo, freddo/caldo",None
